In [1]:
!pip install transformers evaluate accelerate datasets
from huggingface_hub import notebook_login
from transformers import AutoTokenizer

notebook_login()

tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [2]:
from datasets import Dataset
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/WIP/Biblio/biblio2.tsv", sep='\t', encoding="ISO-8859-1").dropna(subset="label").dropna(subset="abstract")
df['label'] = df['label'].astype(int)
df_dataset = df[["abstract","label"]]
dataset = Dataset.from_pandas(df_dataset)
dataset = dataset.remove_columns(["__index_level_0__"])

dataset = dataset.train_test_split(test_size=0.2)

In [3]:
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np

def preprocess_function(examples):
    return tokenizer(examples["abstract"], truncation=True, max_length=512)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

tokenized_bilio = dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")

Map:   0%|          | 0/2788 [00:00<?, ? examples/s]

Map:   0%|          | 0/698 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

id2label = {0: "Not Relevant", 1: "Potentially Interesting"}
label2id = {"Not Relevant": 0, "Potentially Interesting": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext", num_labels=2, id2label=id2label, label2id=label2id
)

training_args = TrainingArguments(
    output_dir="PubMed_interests",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_bilio["train"],
    eval_dataset=tokenized_bilio["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.163648,0.936963
2,No log,0.192788,0.948424
3,0.151700,0.200221,0.951289
4,0.151700,0.172552,0.952722


TrainOutput(global_step=700, training_loss=0.12343711035592216, metrics={'train_runtime': 327.0343, 'train_samples_per_second': 34.1, 'train_steps_per_second': 2.14, 'total_flos': 2312935287685440.0, 'train_loss': 0.12343711035592216, 'epoch': 4.0})

In [ ]:
#what's the optimal strategy to keep fine-tuning ?
# fine tune with only the new items?
# fine tune from the original model with the whole dataset ?
